Get Human-Drawn Image

In [ ]:
!pip install transformers torch diffusers kagglehub datasets pillow
!pip install "numpy>=1.26.0,<2.0.0"

In [ ]:
mport kagglehub, os, random, torch
from datasets import load_dataset
from PIL import Image
from diffusers import DDPMPipeline

In [ ]:
# Load Datsets and find unique images
kaggle_path = kagglehub.dataset_download("subinium/highresolution-anime-face-dataset-512x512")
print(f"Kaggle dataset downloaded to: {kaggle_path}")
hf_dataset = load_dataset("kevinbenoy/anime_random_images")
hf_image_names = set()
for example in hf_dataset['train']:
    if 'image' in example:
        hf_image_names.add(example.get('file_name', ''))
kaggle_images = []
for root, dirs, files in os.walk(kaggle_path):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            full_path = os.path.join(root, file)
            if file not in hf_image_names:
                kaggle_images.append(full_path)
print(f"Found {len(kaggle_images)} unique images not in HF dataset")

100%|██████████| 15.0G/15.0G [11:57<00:00, 22.5MB/s]

Extracting files...


In [ ]:
# Get random unique image
for _ in range(4):
  random_image_path = random.choice(kaggle_images)
  img = Image.open(random_image_path)
  display(img.resize((64, 64)))

Generate AI Image

In [ ]:
# Load Model
repo = "Tomisin05/unconditional-anime-diffusion-model"
model = DDPMPipeline.from_pretrained(repo)
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32
model.to(device)

In [ ]:
# Generate Random AI Image
with torch.no_grad():
        images = model(batch_size=4, num_inference_steps=400).images
for i, img in enumerate(images):
    display(img)